# Calling code as it's called from the command line. When you have to.

Ideally, compiled code you use will have Python _wrappers_, so that you can import Python modules and get the benefits of the compiled code, from Python, with values passed back in Python format, etc; this is exactly what you're getting with `numpy`, for example. Not all code has such wrappers, though, and although writing them yourself is a great idea--that's what `libstempo` (installed for Python2 and Python3 on your container already!) is for `TEMPO2`--sometimes you don't have time. In those cases, you can use Python to call the code as if you'd written out the instructions on the command line, instead of in your code.

`presto`, the very popular pulsar-searching package, does actually have `Python2` wrappers, but here we're going to call it as if from the command line (this is what a lot of PALFA processing did, for example) to demonstrate how you can use the Python `subprocess` module to call code as if from the command line.

In [1]:
#Obviously, we need subprocess
import subprocess

In [3]:
#This isn't Python, we're getting a list of what's in our directory with a Jupyter directive
!ls

Basic_basics.ipynb		     Numpy_and_you.ipynb
Faking_it_at_the_command_line.ipynb  piccard
Functions.ipynb			     puppi_57166_J1404+12_1326_subs_0001.fits
getting_stuff_done.ipynb	     Python2_and_Python3_Some__future__.ipynb
Loops_ifs_files.ipynb		     some_pulsars.txt


## Let's plot ourselves a pulsar

Here we're going to create a plot of pulsar `J1404+12`

In [4]:
#Back to Python, we're setting the filename, which you can see in the list above.
filename = 'puppi_57166_J1404+12_1326_subs_0001.fits'

In [6]:
#We're going to use the Popen functionality in subprocess. You can read some basic documentation with this command
help(subprocess.Popen)


Help on class Popen in module subprocess:

class Popen(builtins.object)
 |  Execute a child program in a new process.
 |  
 |  For a complete description of the arguments see the Python documentation.
 |  
 |  Arguments:
 |    args: A string, or a sequence of program arguments.
 |  
 |    bufsize: supplied as the buffering argument to the open() function when
 |        creating the stdin/stdout/stderr pipe file objects
 |  
 |    executable: A replacement program to execute.
 |  
 |    stdin, stdout and stderr: These specify the executed programs' standard
 |        input, standard output and standard error file handles, respectively.
 |  
 |    preexec_fn: (POSIX only) An object to be called in the child process
 |        just before the child is executed.
 |  
 |    close_fds: Controls closing or inheriting of file descriptors.
 |  
 |    shell: If true, the command will be executed through the shell.
 |  
 |    cwd: Sets the current directory before the child is executed.
 |  
 |   

In [7]:
#Here, we're setting it up to call prepfold from the presto quite to produce output
#from the pulsar in that file with a period of ~2.65s and dm of 18.836
#You can look at presto documentation on Github to see what it all means

p = subprocess.Popen(['prepfold',filename,'-p','2.650585','-dm','18.836','-npart','8','-nsub','128','-noxwin'],stdout=subprocess.PIPE,stderr=subprocess.PIPE)

In [8]:
out, err = p.communicate()

In [10]:
#Let's see if anything's changed!
!ls

Basic_basics.ipynb
Faking_it_at_the_command_line.ipynb
Functions.ipynb
getting_stuff_done.ipynb
Loops_ifs_files.ipynb
Numpy_and_you.ipynb
piccard
puppi_57166_J1404+12_1326_subs_0001_2650.59ms_Cand.pfd
puppi_57166_J1404+12_1326_subs_0001_2650.59ms_Cand.pfd.bestprof
puppi_57166_J1404+12_1326_subs_0001_2650.59ms_Cand.pfd.ps
puppi_57166_J1404+12_1326_subs_0001.fits
Python2_and_Python3_Some__future__.ipynb
some_pulsars.txt


## Did it work?

What you see here are four files produced by `prepfold`. You have `imagemagick` installed on the container, so if you ensure you have an Xserver running on your laptop (`Vcxsrv` or `Xming` for Windows, say, or `XQuartz` for OSX) you can run:

`ssh -Y -p 2222 jovyan@localhost`

to get into the container with X-forwarding, and you can then just run this to get a display of the postscript file:

`display -flatten puppi_57166_J1404+12_1326_subs_0001_2650.59ms_Cand.pfd.ps &`

Now you can see the pulsar in all its glory!

## Let's look inside the file

Here we use `presto`'s readfile.

In [11]:
p = subprocess.Popen(['readfile',filename],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
out, err = p.communicate()
#a b before a string indicates, in Python3, byte literals
result = out.split(b"\n")
for res in result:
    if b"Central freq" in res:
        freq = res.split(b"=")[1].strip()
        print("Frequency=%0.1f" % float(freq))

Frequency=327.0
